
# Datos ausentes

La mayor parte de los datasets presentan registros con uno o varios campos cuya información está ausente (**missing values**), lo que puede generar problemas al intentar representar los datos, realizar ciertas operaciones o aplicarlo a un algoritmo. Es por eso que es necesario identificar y tratar esos valores ausentes.

Las dos estrategias de tratamiento son el borrado o la asignación un valor determinado.

`Pandas` toma a los valores `NaN` y `None` como valores ausentes.



## Identificación de missing values

La primera acción a tomar es determinar si existen valores NaNs o None dentro del dataset.



In [65]:
import pandas as  pd
import numpy as np

In [66]:
# Creamos dataframe  con missing values

df = pd.DataFrame({'VarA': ['aa', None, 'cc',None],
                  'VarB': [20, 30, None,None],
                  'VarC': [1234, 3456, 6789,765],
                  'VarD': [1234, 888, None,None]
                 },
                 index=['Case1', 'Case2', 'Case3', 'Case4'])
df

,VarA,VarB,VarC,VarD
Case1,aa,20.0,1234,1234.0
Case2,None,30.0,3456,888.0
Case3,cc,NaN,6789,NaN
Case4,None,NaN,765,NaN


In [67]:
# Comprobamos si existe algún NaN o None en el dataframe

df.isnull()

,VarA,VarB,VarC,VarD
Case1,False,False,False,False
Case2,True,False,False,False
Case3,False,True,False,True
Case4,True,True,False,True


En el ejemplo anterior es fácil comprobar si existe algún valor ausente, pero con dataframes más grandes es necesario recurrir a otros métodos:

In [68]:
# Nos indica si algún elemento dentro del df es un missing value

df.isnull().values.any()

True

In [69]:
# Comprueba que columnas tienen NaNs

df.isnull().any()

VarA     True
VarB     True
VarC    False
VarD     True
dtype: bool

In [70]:
# Nos indica el número de registros con NaNs por cada columna

df.isnull().sum()

VarA    2
VarB    2
VarC    0
VarD    2
dtype: int64

In [71]:
# Devuelve el número de missing values por cada registro

df.isnull().sum(axis=1)

Case1    0
Case2    1
Case3    2
Case4    3
dtype: int64

## Eliminación de missing values

La estrategia más sencilla de tratamiento de missing values consiste en eliminar los registros que los contengan.


In [72]:
# Eliminamos cualquier registro que contenga al menos un NaN

df.dropna()

,VarA,VarB,VarC,VarD
Case1,aa,20.0,1234,1234.0


Sin embargo, como vemos más abajo en el df no se han eliminado los registros con  NaNs. Para que el cambio se ejecute es necesario usar la opción inplace=True: `df.dropna(inplace=True)`

In [73]:
df

,VarA,VarB,VarC,VarD
Case1,aa,20.0,1234,1234.0
Case2,None,30.0,3456,888.0
Case3,cc,NaN,6789,NaN
Case4,None,NaN,765,NaN


In [74]:
# Podemos indicar un conjunto de columnas en los que eliminar los NaNs.

df.dropna(subset=['VarB'])

,VarA,VarB,VarC,VarD
Case1,aa,20.0,1234,1234.0
Case2,None,30.0,3456,888.0


In [75]:
# Aplicamos la condición de que todos los valores del registro sean NaNs para borrar el registro.

df.dropna(how='all')

,VarA,VarB,VarC,VarD
Case1,aa,20.0,1234,1234.0
Case2,None,30.0,3456,888.0
Case3,cc,NaN,6789,NaN
Case4,None,NaN,765,NaN


In [76]:
# O podemos aplicar un límite con un número mínimo de valores no NaNs por registro.

df.dropna(thresh=3)

,VarA,VarB,VarC,VarD
Case1,aa,20.0,1234,1234.0
Case2,None,30.0,3456,888.0


## Asignación de valores

La otra opción de tratamiento consiste en asignar un valor determinado a las instancias con missing values. Lo hacemos con el método `fillna`, que al igual que dropna requiere
el parámetro inplace=True para persistir los cambios.

In [77]:
df.fillna('new')

,VarA,VarB,VarC,VarD
Case1,aa,20.0,1234,1234.0
Case2,new,30.0,3456,888.0
Case3,cc,new,6789,new
Case4,new,new,765,new


Esta asignación del string 'new' se ha realizado sobre todos los elementos missing values. Para ello Pandas ha realizado un cambio en el tipo de alguna de las variables.

In [78]:
# Comprobamos que el type antes y descués del fillna es distinto para la variable numérica

df['VarB'].dtype, df.fillna('new')['VarB'].dtype

(dtype('float64'), dtype('O'))

Para evitar este tipo de cambios no deseados seleccionamos la columna a modificar.

In [79]:
# Realizamos únicamente el fillna sobre la columna VarA

df['VarA'].fillna('new', inplace=True)
df

,VarA,VarB,VarC,VarD
Case1,aa,20.0,1234,1234.0
Case2,new,30.0,3456,888.0
Case3,cc,NaN,6789,NaN
Case4,new,NaN,765,NaN


In [80]:
# Podemos realizar una asignación de valores tomándo el último valor observado y propagándolo hacia adelante.

df['VarB'].fillna(method='ffill')

Case1    20.0
Case2    30.0
Case3    30.0
Case4    30.0
Name: VarB, dtype: float64

In [81]:
df['VarB']

Case1    20.0
Case2    30.0
Case3     NaN
Case4     NaN
Name: VarB, dtype: float64

In [82]:
# También es posible asignar el resultado de una función, en este caso el valor medio de la columna.

df['VarB'].fillna(np.mean(df['VarB']))

Case1    20.0
Case2    30.0
Case3    25.0
Case4    25.0
Name: VarB, dtype: float64

# Ejercicios

Dado el siguiente Dataframe

In [83]:
raw_data = {'first_name': ['Jason', 'Mary', 'Tina', 'Jake', 'Amy','Anne','Alice'],
        'last_name': ['Miller', 'Smith', 'Ali', 'Milner', 'Cooze','Lynn','Smith'],
        'age': [42, np.nan, 36, 24, 73,'23',20],
        'sex': ['m', np.nan, 'f', 'm', 'f','f','m'],
        'preTestScore': [4, np.nan, np.nan, 2, 3, np.nan,100],
        'postTestScore': [25, np.nan, np.nan, 62, 70, np.nan,65]}
df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age', 'sex', 'preTestScore', 'postTestScore'])

- a. Determine que columna(s) tiene(n) el mayor número de NaNs.
- b. Complete las variables categóricas nulas con el valor mayoritario.
- c. Elimine los registros con mayor número de nulos
- d. Complete la variable 'preTestScore' con el valor medio
- e. Complete la variable 'postTestScore' con el valor inmediatamente posterior


##### Solución Ejercicio 1

Determine que columna(s) tiene(n) el mayor número de NaNs.

In [84]:
df.isnull().sum(axis=0)[df.isnull().sum()==max(df.isnull().sum())]

preTestScore     3
postTestScore    3
dtype: int64

In [85]:
df.isnull().sum(axis=1)

0    0
1    4
2    2
3    0
4    0
5    2
6    0
dtype: int64

In [86]:
df.dtypes[4]

dtype('float64')

Complete las variables categóricas nulas con el valor mayoritario.

In [87]:


df['sex'].fillna(df["sex"].mode().iloc[0])

0    m
1    f
2    f
3    m
4    f
5    f
6    m
Name: sex, dtype: object

In [88]:
?df.fillna

In [89]:
df

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42,m,4.0,25.0
1,Mary,Smith,NaN,NaN,NaN,NaN
2,Tina,Ali,36,f,NaN,NaN
3,Jake,Milner,24,m,2.0,62.0
4,Amy,Cooze,73,f,3.0,70.0
5,Anne,Lynn,23,f,NaN,NaN
6,Alice,Smith,20,m,100.0,65.0


Elimine los registros con mayor número de nulos

In [90]:
df.dropna(thresh=df.shape[1]-max(df.isnull().sum()))

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42,m,4.0,25.0
2,Tina,Ali,36,f,NaN,NaN
3,Jake,Milner,24,m,2.0,62.0
4,Amy,Cooze,73,f,3.0,70.0
5,Anne,Lynn,23,f,NaN,NaN
6,Alice,Smith,20,m,100.0,65.0


Complete la variable 'preTestScore' con el valor medio

In [91]:
df['preTestScore'].mean()

27.25

In [92]:
df['preTestScore'].fillna(df['preTestScore'].mean())#,inplace=True)


0      4.00
1     27.25
2     27.25
3      2.00
4      3.00
5     27.25
6    100.00
Name: preTestScore, dtype: float64

Complete la variable 'preTestScore' con la mediana

In [93]:
df['preTestScore'].median()

3.5

In [94]:
df['preTestScore'].fillna(df['preTestScore'].median())#,inplace=True)


0      4.0
1      3.5
2      3.5
3      2.0
4      3.0
5      3.5
6    100.0
Name: preTestScore, dtype: float64

Complete la variable 'postTestScore' con el valor inmediatamente posterior

In [95]:
# Aqui su respuesta
df['postTestScore'].fillna(method='bfill')#,inplace=True)

0    25.0
1    62.0
2    62.0
3    62.0
4    70.0
5    65.0
6    65.0
Name: postTestScore, dtype: float64